# QA с помощью RAG
В этом ноутбуке мы создадим QA pipeline с langchain-gigachat.
Мы загрузим главную страницу репозитория [gigachat-js](https://github.com/ai-forever/gigachat-js/tree/master)
и поспрашиваем GigaChat вопросы по этой библиотеке.

## Установка
### Зависимости
```bash
npm install --save langchain langchain-gigachat cheerio
```
### Переменные окружения
```bash
export GIGACHAT_CREDENTIALS='ваши креды'
export GIGACHAT_SCOPE='GIGACHAT_API_CORP' или 'GIGACHAT_API_PERS' или 'GIGACHAT_API_B2B'
```

Инициализируем подключение к LLM

In [2]:
import { Agent } from 'node:https';
import { GigaChat } from "langchain-gigachat"
/*
В обычном Node.js такое отключение проверки сертификатов срабатывает.

Но в Deno нет, 
поэтому используйте проставление NODE_EXTRA_CA_CERTS в env.
 */
const httpsAgent = new Agent({
    rejectUnauthorized: false,
}); 

const llm = new GigaChat({
    maxRetries: 0,
    httpsAgent
})

Инициализируем нашу RAG цепочку

In [ ]:
import "cheerio";
import { CheerioWebBaseLoader } from "@langchain/community/document_loaders/web/cheerio";
import { RecursiveCharacterTextSplitter } from "langchain/text_splitter";
import { MemoryVectorStore } from "langchain/vectorstores/memory";
import { GigaChatEmbeddings, GigaChat } from "langchain-gigachat";
import { pull } from "langchain/hub";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { formatDocumentsAsString } from "langchain/util/document";
import {
  RunnableSequence,
  RunnablePassthrough,
} from "@langchain/core/runnables";
import { StringOutputParser } from "@langchain/core/output_parsers";

const loader = new CheerioWebBaseLoader(
  "https://raw.githubusercontent.com/ai-forever/gigachat-js/refs/heads/master/README.md"
);

const docs = await loader.load();

const textSplitter = new RecursiveCharacterTextSplitter({
  chunkSize: 1000,
  chunkOverlap: 200,
});
const splits = await textSplitter.splitDocuments(docs);
const vectorStore = await MemoryVectorStore.fromDocuments(
  splits,
  new GigaChatEmbeddings()
);

const retriever = vectorStore.asRetriever();
const prompt = await pull<ChatPromptTemplate>("gigachat/rag-prompt");

const ragChain = RunnableSequence.from([
  {
    context: retriever.pipe(formatDocumentsAsString),
    question: new RunnablePassthrough(),
  },
  prompt,
  llm,
  new StringOutputParser(),
]);


In [4]:
await ragChain.invoke("Как установить сертификаты при подключении к GigaChat");

UPDATE TOKEN


"Для установки сертификатов при подключении к GigaChat необходимо использовать HTTPS-агент с вашими сертификатами. Вот пример кода на TypeScript:\n" +
  "\n" +
  "```typescript\n" +
  "import GigaChat from 'gigachat';\n" +
  "import { Agent } from 'node:https';\n" +
  "import fs from 'node:fs';\n" +
  "\n" +
  "const httpsAgent = new Agent({\n" +
  "  ca: fs.readFileSync('path/to/your/ca.pem'),\n" +
  "  cert: fs.readFileSync('path/to/your/tls.pem'),\n" +
  "  key: fs.readFileSync('path/to/your/tls.key'),\n" +
  "  passphrase: 'password',\n" +
  "});\n" +
  "\n" +
  "const client = new GigaChat({\n" +
  "  baseUrl: 'YOUR_API_URL',\n" +
  "  httpsAgent: httpsAgent,\n" +
  "});\n" +
  "```\n" +
  "\n" +
  "Не забудьте заменить `path/to/your/` на реальные пути к вашим сертификатам."

Хорошо, но при получении ответа, хотелось бы, чтобы возвращались куски документа, которые использовались при формировании ответа
# Добавление sources

In [9]:
import {
    RunnableMap,
    RunnablePassthrough,
    RunnableSequence,
  } from "@langchain/core/runnables";
  import { formatDocumentsAsString } from "langchain/util/document";
  
  const ragChainWithSources = RunnableMap.from({
    // Return raw documents here for now since we want to return them at
    // the end - we'll format in the next step of the chain
    context: retriever,
    question: new RunnablePassthrough(),
  }).assign({
    answer: RunnableSequence.from([
      (input) => {
        return {
          // Now we format the documents as strings for the prompt
          context: formatDocumentsAsString(input.context),
          question: input.question,
        };
      },
      prompt,
      llm,
      new StringOutputParser(),
    ]),
  });
  
  let resp1 = await ragChainWithSources.invoke("Через какой параметр прописывается токен подключения");

In [11]:
resp1['answer']

"Токен подключения прописывается через параметр `credentials` при инициализации объекта `GigaChat`."

Смотри какие документы использовались при формировании ответа

In [12]:
resp1['context']

[
  Document {
    pageContent: "| `timeout`                 | Таймаут (в секундах), который используется при подключении                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |",
    metadata: {
      source: "https://raw.githubusercontent.com/ai-forever/gigachat-js/refs/heads/master/README.md",
      loc: { lines: { from: 154, to: 154 } }
    },
    id: undefined
  },
  Document {
    pageContent: "## Способы авторизации\n" +
      "\n" +
      "Для авторизации запросов, кроме ключа, полученного в личном кабинете, вы можете использовать:\n" +
      "\n" +
      "- crede